In [5]:
#installing dependencies/imports
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from PIL import Image
import os
import shutil

In [6]:
pip install numpy


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
pip install matplotlib


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
folder_path_test = "C:\\Users\\Sayeed\\OneDrive\\Documents\\Python Scripts\\eyediseasep2\\dataset\\cataract"
source = "C:\\Users\\Sayeed\\OneDrive\\Documents\\Python Scripts\\eyediseasep2\\dataset"
data_list = os.listdir(folder_path_test)
for i in data_list:
    print(i)

0_left.jpg
103_left.jpg
1062_right.jpg
1083_left.jpg
1084_right.jpg
1102_left.jpg
1102_right.jpg
1115_left.jpg
1126_right.jpg
112_right.jpg
1144_left.jpg
1144_right.jpg
1164_left.jpg
1167_right.jpg
119_left.jpg
1285_left.jpg
1415_left.jpg
1415_right.jpg
1435_left.jpg
1444_left.jpg
1452_left.jpg
1454_right.jpg
1456_left.jpg
1460_right.jpg
1468_left.jpg
1483_right.jpg
1484_left.jpg
1484_right.jpg
1490_right.jpg
188_right.jpg
1968_left.jpg
1987_left.jpg
1994_left.jpg
2052_left.jpg
2052_right.jpg
2074_right.jpg
2095_right.jpg
2096_right.jpg
2097_right.jpg
2098_left.jpg
2098_right.jpg
2099_left.jpg
2100_right.jpg
2101_left.jpg
2101_right.jpg
2103_left.jpg
2103_right.jpg
2104_left.jpg
2104_right.jpg
2105_left.jpg
2105_right.jpg
2106_right.jpg
2107_left.jpg
2108_left.jpg
2108_right.jpg
2109_left.jpg
2109_right.jpg
2110_left.jpg
2110_right.jpg
2111_left.jpg
2113_left.jpg
2113_right.jpg
2114_left.jpg
2114_right.jpg
2115_right.jpg
2116_left.jpg
2116_right.jpg
2117_left.jpg
2118_right.jpg
2119_ri

In [14]:
train_folder = "C:\\Users\\Sayeed\\OneDrive\\Documents\\Python Scripts\\eyediseasep2\\train_folder"
val_folder = "C:\\Users\\Sayeed\\OneDrive\\Documents\\Python Scripts\\eyediseasep2\\val_folder"

os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)

In [15]:
class_labels = os.listdir(source)

def create_class_folders(parent_folder):
    for label in class_labels:
        class_folder = os.path.join(parent_folder, label)
        os.makedirs(class_folder, exist_ok=True)

create_class_folders(train_folder)
create_class_folders(val_folder)


In [16]:
import random

In [20]:
train_percent = 0.8
val_percent = 0.2

for class_label in os.listdir(source):
    class_folder = os.path.join(source, class_label)
    files = os.listdir(class_folder)

    num_train_files = int(len(files) * train_percent)
    
    #kind of unesscary shuffle but solves any potential mistakes later with sequential data
    random.shuffle(files)
    
    train_files = files[:num_train_files]
    
    for file in train_files:
        source_curr = os.path.join(class_folder, file)
        dest = os.path.join(train_folder, class_label, file)
        os.makedirs(os.path.dirname(dest), exist_ok=True)
        shutil.copy(source_curr, dest)

In [21]:
print(len(os.listdir(folder_path_test)))
print(len(os.listdir(os.path.join(train_folder, "cataract"))))
#perfect split of 80/20

1038
830


In [22]:
for class_label in os.listdir(source):
    class_folder = os.path.join(source, class_label)
    files = os.listdir(class_folder)

    num_train_files = int(len(files) * val_percent)
    
    #kind of unesscary shuffle but solves any potential mistakes later with sequential data
    random.shuffle(files)
    
    train_files = files[:num_train_files]
    
    for file in train_files:
        source_curr = os.path.join(class_folder, file)
        dest = os.path.join(val_folder, class_label, file)
        os.makedirs(os.path.dirname(dest), exist_ok=True)
        shutil.copy(source_curr, dest)

In [23]:
img = "C:\\Users\\Sayeed\\OneDrive\\Documents\\Python Scripts\\eyediseasep2\\train_folder\\cataract\\cataract_087.png"
image = Image.open(img)
width, height = image.size

print("Width:", width)
print("Height:", height)

Width: 2592
Height: 1728


In [24]:
img_height = 256
img_width = 256
num_channels = 3
num_classes = 4

train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=20,  
    width_shift_range=0.2,  
    height_shift_range=0.2,
    shear_range=0.2, 
    zoom_range=0.2, 
    fill_mode='nearest' 
)

val_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 24

train_generator = train_datagen.flow_from_directory(
    train_folder, 
    target_size=(img_height, img_width),
    batch_size=batch_size,  
    class_mode='categorical' 
)
train_generator = train_generator.repeat()

val_generator = val_datagen.flow_from_directory(
    val_folder,  
    target_size=(img_height, img_width), 
    batch_size=batch_size, 
    class_mode='categorical' 
)
val_generator = val_generator.repeat()


Found 3372 images belonging to 4 classes.
Found 841 images belonging to 4 classes.


In [30]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, num_channels)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu'),  
    MaxPooling2D((2, 2)),  
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.summary()


c:\Users\Sayeed\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 12, 12, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │     4,719,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │         2,052 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,257,156 (20.05 MB)

 Trainable params: 5,257,156 (20.05 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [27]:
import sys
from PIL import Image
sys.modules['Image'] = Image 

In [ ]:
pip install --upgrade tensorflow keras numpy pandas scikit-learn pillow


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/ab/63/966db1321a0ad55df1d1fe51505d2cdae191b84c907974873817b0a6e849/pandas-2.2.2-cp311-cp311-win_amd64.whl.metadata
  Using cached pandas-2.2.2-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/79/3d/02d5d3ed359498fec3abdf65407d3c07e3b8765af17464969055aaec5171/scikit_learn-1.4.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/9c/3d/a121f284241f08268b21359bd425f7d4825cffc5ac5cd0e1b3d82ffd2b10/pytz-2024.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.org/packages/65/58/f9c9e6be752e9fcb8b6a0ee9fb87e6e7a1f6bcab2cdc73f02bb7ba91ada0/tzdata-2024.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for scipy>=1.6.0 from https://files.pythonhosted.org/

In [35]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20, 
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size
)

Epoch 1/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 111s 770ms/step - accuracy: 0.8329 - loss: 0.4082 - val_accuracy: 0.8036 - val_loss: 0.4408
Epoch 2/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 209us/step - accuracy: 0.8333 - loss: 0.3559 - val_accuracy: 1.0000 - val_loss: 0.0281
Epoch 3/20


c:\Users\Sayeed\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


140/140 ━━━━━━━━━━━━━━━━━━━━ 101s 702ms/step - accuracy: 0.8444 - loss: 0.3967 - val_accuracy: 0.8464 - val_loss: 0.3855
Epoch 4/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 205us/step - accuracy: 1.0000 - loss: 0.1271 - val_accuracy: 1.0000 - val_loss: 0.0168
Epoch 5/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 100s 697ms/step - accuracy: 0.8550 - loss: 0.3785 - val_accuracy: 0.8357 - val_loss: 0.3912
Epoch 6/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 223us/step - accuracy: 0.7917 - loss: 0.4785 - val_accuracy: 0.0000e+00 - val_loss: 0.7555
Epoch 7/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 98s 682ms/step - accuracy: 0.8421 - loss: 0.3903 - val_accuracy: 0.8321 - val_loss: 0.3790
Epoch 8/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 194us/step - accuracy: 0.9167 - loss: 0.2252 - val_accuracy: 1.0000 - val_loss: 0.0144
Epoch 9/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 98s 682ms/step - accuracy: 0.8453 - loss: 0.3857 - val_accuracy: 0.8595 - val_loss: 0.3367
Epoch 10/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 201us/step - accuracy: 0.8750 - loss: 0.2156 - 

KeyboardInterrupt: 